# Part 1:

scrapping data from wikipedia:


In [57]:
import requests
import pandas as pd


In [58]:
from bs4 import BeautifulSoup


In [59]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

# All requested packages already installed.



In [60]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done

# All requested packages already installed.



In [61]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
Page= requests.get(url).text

In [62]:
Bsoup= BeautifulSoup(Page,'lxml')
TargetTabel=Bsoup.find('table')

In [63]:
table = pd.read_html(Page)
df = pd.DataFrame(table[0])
df.columns=['Postcode','Borough','Neighbourhood']
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [64]:
#Delete 'Not Assigned' values from the dataframe
df=df[df.Borough != 'Not assigned']
UpdatedDF=pd.DataFrame(df)

#Combined Neighbourhoods at the same Postcode
UpdatedDF2=df.groupby("Postcode").agg(lambda x:','.join(set(x)))
UpdatedDF2=UpdatedDF2.reset_index()
UpdatedDF2.head(20)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Port Union,Rouge Hill"
2,M1E,Scarborough,"Morningside,West Hill,Guildwood"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Ionview,East Birchmount Park,Kennedy Park"
7,M1L,Scarborough,"Oakridge,Golden Mile,Clairlea"
8,M1M,Scarborough,"Cliffside,Cliffcrest,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


# Part 2:

In [65]:
url="https://cocl.us/Geospatial_data"
Gdata=pd.read_csv(url)
Gdata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [66]:
#Extract the last two columns
NewGdata= Gdata[Gdata.columns[-2:]]
NewGdata.head()


,Latitude,Longitude
0,43.806686,-79.194353
1,43.784535,-79.160497
2,43.763573,-79.188711
3,43.770992,-79.216917
4,43.773136,-79.239476


In [67]:
# Appen our first dataframe with the Latitude and Longitude
UpdatedDF2 = pd.concat([UpdatedDF2,NewGdata], axis=1,verify_integrity=False)
UpdatedDF2.head()



,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Port Union,Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Morningside,West Hill,Guildwood",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# part 3 :

In [68]:
address = 'Torronto, CA'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Torronto are {}, {}.'.format(latitude, longitude))

# 43.6532° N, 79.3832° W


AttributeError: 'NoneType' object has no attribute 'latitude'

In [69]:
#Creat a map:

import folium

map_Toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(UpdatedDF2['Latitude'], UpdatedDF2['Longitude'], UpdatedDF2['Borough'], UpdatedDF2['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  

    
    #It shoud be worked!


KeyError: 'Neighborhood'